In [ ]:
import pandas as pd
data=pd.read_csv(r'C:\Users\svnduw\Desktop\ML\all_data.csv')

In [ ]:
data['date'] = pd.to_datetime(data['date'])

In [ ]:
data['period']=pd.to_datetime(data['date']).dt.to_period('M')


In [ ]:
tobacoo=pd.read_excel(r'C:\Users\svnduw\Desktop\ML\umag_report_makenov_barcode_information.xlsx')

In [ ]:
tobacoo_barcode=tobacoo.barcode.tolist()

In [ ]:
len(data)

In [ ]:
data=data[data['barcode'].isin(tobacoo_barcode)]

In [ ]:
len(data)

In [ ]:
new=data.groupby(by=['period','store_id','barcode']).sum()[['product_quantity','total_sale_amount']].reset_index()

# Get temperature

In [ ]:
from bs4 import BeautifulSoup
from datetime import date
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from selenium.webdriver.chrome.options import Options

In [ ]:
options = Options()
options.add_argument("--disable-notifications")
driver = webdriver.Chrome(executable_path='C:\\Users\\svnduw\\.wdm\\drivers\\chromedriver\\win32\\93.0.4577.15\\chromedriver.exe', chrome_options=options)
driver.get('https://weather.com/weather/monthly/l/fbaf85b9d5c9c522f7fafeb07232ae25af12eb0dd5756a22afd041e7adb97f2f')
time.sleep(5)

In [ ]:
l=[8]

In [ ]:
for j in range(len(l)):    
    driver.find_element_by_xpath('//*[@id="years-picker"]/option[2]').click()#2020=1,2021=2 in option
    driver.find_element_by_xpath('//*[@id="month-picker"]/option['+str(l[j])+']').click()#month
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="WxuCalendar-main-69bb61a5-2a5f-4a3e-879c-53f37030c238"]/section/div[2]/nav/div/button/span').click()
    time.sleep(5)    
    a=[]
    for i in range(len(driver.find_elements_by_xpath('//*[@id="WxuCalendar-main-69bb61a5-2a5f-4a3e-879c-53f37030c238"]/section/div[3]/button'))):
        a.append(driver.find_element_by_xpath('//*[@id="WxuCalendar-main-69bb61a5-2a5f-4a3e-879c-53f37030c238"]/section/div[3]/button['+str(i+1)+']').text.split('\n'))
    list_name=driver.find_element_by_xpath('//*[@id="month-picker"]/option['+str(l[j])+']').text+' '+driver.find_element_by_xpath('//*[@id="years-picker"]/option[2]').text
    f = pd.DataFrame(a,columns =['Day','Day_temperature','Night_temperature'])
    writer=pd.ExcelWriter(r"C:\Users\svnduw\Desktop\ML\Temperature_base\Almaty\\"+list_name+'.xlsx',engine = 'xlsxwriter')
    f.to_excel(writer,index=False,sheet_name=list_name)
    writer.save()
    writer.close()

# ###########################################

In [ ]:
from os import listdir
from os.path import isfile, join
mypath=r'C:\Users\svnduw\Desktop\ML\Temperature_base\Almaty\checked\\'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
result=pd.DataFrame()
for i in onlyfiles:
    if i.endswith('.xlsx'):
        a=pd.read_excel(r'C:\Users\svnduw\Desktop\ML\Temperature_base\Almaty\checked\\'+i)
        a['Day_temperature']=a['Day_temperature'].str.replace('°','')
        a['Night_temperature']=a['Night_temperature'].str.replace('°','')
        a['Day_temperature']=a['Day_temperature'].astype(int)
        a['Night_temperature']=a['Night_temperature'].astype(int)
        a['Day_temperature(C)']=(a['Day_temperature']-32)*5//9
        a['Night_temperature(C)']=(a['Night_temperature']-32)*5//9
        a['source']=i
        result=result.append(a,ignore_index=True) 
        

In [ ]:
result.to_excel(r'C:\Users\svnduw\Desktop\ML\Temperature_base\Almaty\combined.xlsx',index=False)

# ###########################################

In [ ]:
import pandas as pd
import numpy as np
import datetime

result.source=result.source.str.replace('.xlsx','')
result["Date"]=result["Day"].astype(str) +' '+result["source"]

In [ ]:
###result=pd.read_csv(r'C:\Users\svnduw\Desktop\ML\Final\result.csv')

In [ ]:
result['Date']=result['Date'].apply(lambda x: datetime.datetime.strptime(x, '%d %b %Y').strftime('%Y-%m-%d'))

In [ ]:
result['period']=pd.to_datetime(result['Date']).dt.to_period('M')

In [ ]:
temp=result.groupby(by=['period']).mean()[['Day_temperature(C)','Night_temperature(C)']].reset_index()

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type =df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float64).min and c_max < np.finfo(np.float64).max:
                    df[col] = df[col].astype(np.float64)
        end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100*(start_mem-end_mem)/start_mem))
    return df

In [ ]:
data = reduce_mem_usage(data, verbose=True)

In [ ]:
data['date'] = pd.to_datetime(data['date'])

In [ ]:
result['Date']=pd.to_datetime(result['Date'])

In [ ]:
temp

In [ ]:
new.head()

In [ ]:
temp.columns = ['Day', 'day_temperature(C)', 'night_temperature(C)']

In [ ]:
new['day_temperature(C)']=new['period'].map(dict(zip(temp['Day'],temp['day_temperature(C)'])))
new['night_temperature(C)']=new['period'].map(dict(zip(temp['Day'],temp['night_temperature(C)'])))


In [ ]:
data.drop('month',axis=1,inplace=True)

In [ ]:
data['brand']=data['barcode'].map(dict(zip(tobacoo['barcode'],tobacoo['brand'])))

In [ ]:
holiday_2020=pd.read_excel(r'C:\Users\svnduw\Desktop\ML\Holiday\2020\holidays_2020.xlsx')
holiday_2021=pd.read_excel(r'C:\Users\svnduw\Desktop\ML\Holiday\2021\holidays_2021.xlsx')

In [ ]:
day_2020=holiday_2020['date'].tolist()
day_2021=holiday_2021['date'].tolist()

In [ ]:
day_2020.extend(day_2021)

In [ ]:
temp=data[data['date'].isin(day_2020)]
temp2=data[~data['date'].isin(day_2020)]
temp['isHoliday']=1
temp2['isHoliday']=0


In [ ]:
new=temp.append(temp2,ignore_index=True)

In [ ]:
new=new.sort_values(by=['date'])

In [ ]:
result.isHoliday.value_counts()

In [ ]:
result["isHoliday"]=result["isHoliday"].astype(int)

# #############################################

In [ ]:
import pandas as pd
data_file=pd.read_excel(r'C:\Users\svnduw\Desktop\ML\актуализация адресов copy.xlsx')

In [ ]:
data_file=data_file[['Store_Id','Link YandexMap(Сылка от Яндекс Карта )']]

In [ ]:
b=data_file['Store_Id'].tolist()

In [ ]:
found=[]
for i in a:
    if i in b:
        found.append(i)

In [ ]:
url=data_file[data_file['Store_Id'].isin(found)]['Link YandexMap(Сылка от Яндекс Карта )'].tolist()

In [ ]:
url=data_file[data_file['Link YandexMap(Сылка от Яндекс Карта )'].notnull()]['Link YandexMap(Сылка от Яндекс Карта )'].tolist()

In [ ]:
yandex_url=[]
for i in url:
    if i.startswith('https://yandex.com/maps/') or i.startswith('https://yandex.kz') :
        yandex_url.append(i)
    

In [ ]:
len(yandex_url)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
driver = webdriver.Chrome(executable_path='C:\\Users\\svnduw\\.wdm\\drivers\\chromedriver\\win32\\93.0.4577.15\\chromedriver.exe')
yandex={}
for i in url:
    try:
        driver.get(i)
        time.sleep(1)
        y=driver.current_url
        if len(y)<100:
            y_lon=y.split('?ll=')[1].split('%2C')[0]
            y_lat=y.split('?ll=')[1].split('%2C')[1].split('&')[0]
            yandex[i]=str(y_lon)+' | '+str(y_lat)+' | '+str(driver.find_element_by_class_name('toponym-card-title-view__description').text)+' | '+str(driver.find_element_by_class_name('card-title-view__title').text)
        else:
            y_lon=y.split('ll=')[1].split('%2C')[0]
            y_lat=y.split('?ll=')[1].split('%2C')[1].split('&')[0]
            yandex[i]=str(y_lon)+' | '+str(y_lat)+' | '+str(driver.find_element_by_class_name('toponym-card-title-view__description').text)+' | '+str(driver.find_element_by_class_name('card-title-view__title').text)
    except NoSuchElementException:
        continue
    

In [ ]:
dataframe=pd.DataFrame(yandex.items(),columns=['yandex_link','information'])


In [ ]:
dataframe[['lon','lat','location','address']] = dataframe['information'].str.split("|",expand=True)

In [ ]:
dataframe.drop('information',axis=1,inplace=True)

In [ ]:
dataframe['store_id']=dataframe['yandex_link'].map(dict(zip(data_file['Link YandexMap(Сылка от Яндекс Карта )'],data_file['Store_Id'])))

In [ ]:
dataframe.to_excel(r'C:\Users\svnduw\Desktop\ML\Address\address3.xlsx')

In [ ]:
data_file['lon']=data_file['Link YandexMap(Сылка от Яндекс Карта )'].map(dict(zip(dataframe['yandex_link'],dataframe['lon'])))
data_file['lat']=data_file['Link YandexMap(Сылка от Яндекс Карта )'].map(dict(zip(dataframe['yandex_link'],dataframe['lat'])))

# ######################################

In [ ]:
address=pd.read_excel(r'C:\Users\svnduw\Desktop\ML\Address\address.xlsx')

In [ ]:
new['longitude']=new['store_id'].map(dict(zip(address['store_id'],address['lon'])))
new['latitude']=new['store_id'].map(dict(zip(address['store_id'],address['lat'])))

In [ ]:
new['brand']=new['barcode'].map(dict(zip(tobacoo['barcode'],tobacoo['brand'])))

In [ ]:
c=list(new['period'].astype(str).unique())

In [ ]:
c = sorted(c)

In [ ]:
c

In [ ]:
list(range(len(c)))

In [ ]:
d = dict(zip(c, list(range(len(c)))))

In [ ]:
new['num_month']=new['period'].astype(str).map(d)

In [ ]:
new.drop('num_month',axis=1,inplace=True)

In [ ]:
temp=new.groupby(by=['barcode']).mean()[['product_quantity','total_sale_amount']].reset_index()

In [ ]:
new['avg_year_quantity']=new['barcode'].map(dict(zip(temp['barcode'],temp['product_quantity'])))
new['avg_year_sale']=new['barcode'].map(dict(zip(temp['barcode'],temp['total_sale_amount'])))

In [ ]:
temp=new[new['longitude'].isnull()]
temp2=new[~new['longitude'].isnull()]

In [ ]:
new_address=pd.read_excel(r'C:\Users\svnduw\Desktop\ML\Address\umag_report_makenov_store_information.xlsx')

In [ ]:
new_address=new_address[['store_id','coordinates']]

In [ ]:
new_address['latitude']=new_address['coordinates'].apply(lambda x: x.split(',')[0])
new_address['longitude']=new_address['coordinates'].apply(lambda x: x.split(',')[1])

In [ ]:
import re
new_address['longitude']=new_address['longitude'].apply(lambda x: re.sub(r"[\([{})\]]", "", x))
new_address['latitude']=new_address['latitude'].apply(lambda x: re.sub(r"[\([{})\]]", "", x))

In [ ]:
new_address['longitude']=new_address['longitude'].astype(float)
new_address['latitude']=new_address['latitude'].astype(float)

In [ ]:
new_address

In [ ]:
temp['longitude']=temp['store_id'].map(dict(zip(new_address['store_id'],new_address['longitude'])))
temp['latitude']=temp['store_id'].map(dict(zip(new_address['store_id'],new_address['latitude'])))

In [ ]:
new3=temp.append(temp2,ignore_index=True).sort_values('period')

In [ ]:
new3['price']=new3['total_sale_amount']/new3['product_quantity']

In [ ]:
new3.columns

In [ ]:
lll=['period', 'store_id', 'barcode','brand', 'product_quantity','price',
       'total_sale_amount', 'avg_year_quantity', 'avg_year_sale','day_temperature(C)', 'night_temperature(C)',
       'longitude', 'latitude']
col=list(new3.columns)
t=[]
for i in lll:
    if i in col:
        t.append(i)
new3=new3[t]

In [ ]:
new3.to_csv(r'C:\Users\svnduw\Desktop\ML\Final\final.csv')

In [ ]:
holidays_period=pd.read_excel(r'C:\Users\svnduw\Desktop\ML\Holiday\holidays\holidays_period.xlsx')

In [ ]:
holidays_period['period'] = pd.to_datetime(holidays_period['period'])

In [ ]:
holidays_period['period']=pd.to_datetime(holidays_period['period']).dt.to_period('M')

In [ ]:
new3['holidays']=new3['period'].map(dict(zip(holidays_period['period'],holidays_period['holidays'])))
new3['weekdays']=new3['period'].map(dict(zip(holidays_period['period'],holidays_period['weekdays'])))

In [ ]:
new=new3.copy()

# ######################################

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
new.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
corr_matrix = new.corr()
corr_matrix["total_sale_amount"].sort_values(ascending=False)

In [ ]:
new.describe().T

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,6))
sns.distplot(new['barcode'])

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot(new['store_id'])

In [ ]:
plt.figure(figsize=(18,6))
plt.subplot(1,2,1)
sns.histplot(new['store_id'])
plt.subplot(1,2,2)
sns.histplot(new['barcode'])

In [ ]:
new.head()

In [ ]:
###new["body_style_cat"] = new["brand"].cat.codes


In [ ]:
def categorize(table):
    table['brand_category']=''
    table.loc[table['brand']== 'Bond','brand_category']=0
    table.loc[table['brand']== 'Camel','brand_category']=1
    table.loc[table['brand']== 'Captain Black','brand_category']=2
    table.loc[table['brand']== 'Cigaronne','brand_category']=3
    table.loc[table['brand']== 'Corsar','brand_category']=4
    table.loc[table['brand']== 'Davidoff','brand_category']=5
    table.loc[table['brand']== 'Dunhill','brand_category']=6
    table.loc[table['brand']== 'Esse','brand_category']=7
    table.loc[table['brand']== 'FAST','brand_category']=8
    table.loc[table['brand']== 'FIIT','brand_category']=9
    table.loc[table['brand']== 'Glamour','brand_category']=10
    table.loc[table['brand']== 'HARVEST','brand_category']=11
    table.loc[table['brand']== 'HEETS','brand_category']=12
    table.loc[table['brand']== 'IQOS','brand_category']=13
    table.loc[table['brand']== 'Jade','brand_category']=14
    table.loc[table['brand']== 'Kent','brand_category']=15
    table.loc[table['brand']== 'Kiss','brand_category']=16
    table.loc[table['brand']== 'L&M','brand_category']=17
    table.loc[table['brand']== 'LD','brand_category']=18
    table.loc[table['brand']== 'LIL SOLID','brand_category']=19
    table.loc[table['brand']== 'Lark','brand_category']=20
    table.loc[table['brand']== 'MARLBORO','brand_category']=21
    table.loc[table['brand']== 'Marlboro','brand_category']=22
    table.loc[table['brand']== 'Muratti','brand_category']=23
    table.loc[table['brand']== 'PEPE','brand_category']=24
    table.loc[table['brand']== 'Pall Mall','brand_category']=25
    table.loc[table['brand']== 'Parliament','brand_category']=26
    table.loc[table['brand']== 'Richmond','brand_category']=27
    table.loc[table['brand']== 'Rothmans','brand_category']=28
    table.loc[table['brand']== 'Sobranie','brand_category']=29
    table.loc[table['brand']== 'Sovereign','brand_category']=30
    table.loc[table['brand']== 'State Line','brand_category']=31
    table.loc[table['brand']== 'Tonino Lamborghini','brand_category']=32
    table.loc[table['brand']== 'Vogue','brand_category']=33
    table.loc[table['brand']== 'West','brand_category']=34
    table.loc[table['brand']== 'Wild Tail','brand_category']=35
    table.loc[table['brand']== 'Winston','brand_category']=36
    table.loc[table['brand']== 'glo','brand_category']=37
    table.loc[table['brand']== 'neo','brand_category']=38
    table.loc[table['brand']== 'ДОНСКОЙ ТАБАК ТЁМНЫЙ','brand_category']=39
    table.loc[table['brand']== 'КОРОНА','brand_category']=40
    table.loc[table['brand']== 'Русский Стиль','brand_category']=41
    table.loc[table['brand']== 'ҚАЗАҚСТАН','brand_category']=42
    return table    

In [ ]:
#new['period']=pd.to_datetime(new['period'])
new['month']=new['period'].dt.month
new['year']=new['period'].dt.year


In [ ]:
new.columns

In [ ]:
X=new[['store_id','barcode','price','avg_year_quantity','avg_year_sale','day_temperature(C)',
      'night_temperature(C)', 'longitude', 'latitude','holidays', 'weekdays', 'brand_category', 'month', 'year']]
y=new[['product_quantity','total_sale_amount']]

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)

In [ ]:
dtr.score(X_test,y_test)

In [ ]:
dtr.predict([[2983,46084286,476.842105,8.773520,4314.345439,14.645161,3.225806,71.452870,51.187909,0,9,18,10,2020]])

In [ ]:
y_test.head()

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
svr = SVR(epsilon=0.2)

In [ ]:
mor = MultiOutputRegressor(svr)

In [ ]:
mor.fit(X_train, y_train)